In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../Dataset/Laplace's Demon.csv")
df.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0


In [3]:
df.dtypes

date          object
area          object
temp         float64
atemp        float64
humidity     float64
windspeed    float64
rain           int64
dtype: object

In [4]:
df.shape

(992, 7)

In [5]:
train_set = df.loc[df['atemp']>0]
train_set.head()

,date,area,temp,atemp,humidity,windspeed,rain
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0
7,2023-01-11,Goldenrod City,25.149444,19.481239,64.818155,11.085351,1
8,2023-01-11,Viridian Forest,25.986556,2.299876,48.217095,2.153007,1


In [6]:
test_set = df.loc[df['atemp']<=0]
test_set.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
5,2023-01-11,Mt. Mortar,24.389659,-77.019645,54.324315,10.642887,1
6,2023-01-28,Olivine City,21.530216,-2.520045,59.038512,19.818355,1
9,2023-01-27,Mt. Mortar,30.257600,-41.769718,42.568695,15.386586,1


In [7]:
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

In [8]:
df.columns

Index(['date', 'area', 'temp', 'atemp', 'humidity', 'windspeed', 'rain'], dtype='object')

In [10]:
col = ['temp', 'humidity', 'windspeed', 'rain']
model = LinearRegression().fit(train_set[col], train_set['atemp'])
pred_y = model.predict(test_set[col])

In [12]:
mean_=train_set['atemp'].mean()
std_ = train_set['atemp'].std()
print(mean_, std_)

46.392572883794614 34.362970829666686


In [14]:
y=norm.ppf(0.25, mean_, std_)

In [15]:
M=((y-pred_y)**2).mean()

In [16]:
M = M**0.5

In [17]:
answer = round(M, 2)
answer

13.56

In [18]:
df.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0


In [21]:
sub2 = df.copy()
sub2['rain'] = np.where(sub2['rain']==0, -1, sub2['rain'])
sub2.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,-1
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,-1


In [26]:
areas = sub2['area'].unique()
len(areas)

32

In [48]:
area = 'Cerulean City'
df_area = sub2.loc[sub2['area']==area]
df_area = df_area.sort_values(by='date')
df_area=df_area.reset_index(drop=True)
df_area['cum_pre'] = df_area['rain']
for i in range(1, len(df_area)):
    df_area.loc[i, 'cum_pre'] = df_area.loc[i-1, 'cum_pre']+df_area.loc[i, 'rain']
min_=df_area['cum_pre'].min()
max_=df_area['cum_pre'].max()
diff = max_-min_
print(min_, max_, diff)

-6 2 8


In [49]:
def getDiff(area):
    df_area = sub2.loc[sub2['area']==area]
    df_area = df_area.sort_values(by='date')
    df_area=df_area.reset_index(drop=True)
    df_area['cum_pre'] = df_area['rain']
    for i in range(1, len(df_area)):
        df_area.loc[i, 'cum_pre'] = df_area.loc[i-1, 'cum_pre']+df_area.loc[i, 'rain']
    min_=df_area['cum_pre'].min()
    max_=df_area['cum_pre'].max()
    diff = max_-min_
    return diff

In [52]:
diffs=[]
for area in areas:
    diff = getDiff(area)
    diffs.append(diff)
    print(area, diff)

Cerulean City 8
Seafoam Islands 6
Goldenrod City 6
Cianwood City 8
Mt. Mortar 11
Olivine City 9
Viridian Forest 9
Mt. Moon 10
Mahogany Town 6
Ruins of Alph 6
Whirl Islands 4
Lavender Town 6
Mt. Silver 14
Tohjo Falls 9
Saffron City 5
Cherrygrove City 9
Pallet Town 6
Pewter City 6
Cinnabar Island 12
Power Plant 7
Vermillion City 10
Blackthorn City 10
Azalea Town 8
Viridian City 6
Ecruteak City 10
New Bark Town 11
Celadon City 16
Safari Zone 13
Fuchsia City 6
Lake of Rage 18
Violet City 8
National Park 7


In [55]:
diffs.sort()
diffs

[4,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 12,
 13,
 14,
 16,
 18]

In [56]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
random_state=229

In [66]:
df_sub3 = df.copy()
df_sub3.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0


In [58]:
df_sub3.columns

Index(['date', 'area', 'temp', 'atemp', 'humidity', 'windspeed', 'rain'], dtype='object')

In [67]:
scaler = StandardScaler().fit(df_sub3[['temp', 'atemp', 'humidity', 'windspeed']])
df_sub3[['temp', 'atemp', 'humidity', 'windspeed']]=scaler.transform(df_sub3[['temp', 'atemp', 'humidity', 'windspeed']])

In [68]:
df_sub3.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,0.524821,-0.604300,-1.250768,-0.859196,1
1,2023-01-17,Cerulean City,-0.895741,-0.524980,1.413644,0.740058,1
2,2023-01-31,Seafoam Islands,0.524496,-0.410293,-1.968393,1.802559,1
3,2023-01-31,Goldenrod City,0.099571,1.767447,0.009146,-0.578371,0
4,2023-01-23,Cianwood City,1.170677,1.104104,-0.770716,-0.029317,0


In [69]:
col = ['temp', 'atemp', 'humidity', 'windspeed', 'rain']
model = KMeans(n_clusters=2, random_state=random_state).fit(df_sub3[col])

In [79]:
df_sub3['label'] = model.labels_
df_sub3['atemp_orig'] = df['atemp']
df_sub3.head()

,date,area,temp,atemp,humidity,windspeed,rain,label,temp_orig,atemp_orig
0,2023-01-13,Cerulean City,0.524821,-0.604300,-1.250768,-0.859196,1,1,26.869606,-8.894446
1,2023-01-17,Cerulean City,-0.895741,-0.524980,1.413644,0.740058,1,1,18.385474,-4.988309
2,2023-01-31,Seafoam Islands,0.524496,-0.410293,-1.968393,1.802559,1,1,26.867670,0.659525
3,2023-01-31,Goldenrod City,0.099571,1.767447,0.009146,-0.578371,0,0,24.329853,107.903405
4,2023-01-23,Cianwood City,1.170677,1.104104,-0.770716,-0.029317,0,0,30.726903,75.236753


In [80]:
df_sub3.groupby('label')['atemp_orig'].mean()

label
0    63.328541
1   -12.434458
Name: atemp_orig, dtype: float64

In [81]:
answer = df_sub3.groupby('label')['atemp_orig'].mean().sum()

In [82]:
answer

50.89408263555124

In [83]:
answer = 50.89